# 🔥 California Fire Model - Colab Setup

**Run this notebook FIRST to set up your Colab environment.**

This will:
1. Mount Google Drive
2. Copy data to local SSD (MUCH faster than Drive)
3. Install dependencies
4. Verify GPU and data

## Step 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ Google Drive mounted!")

## Step 2: Check GPU

**Make sure you're on an A100!**
- Go to Runtime → Change runtime type → GPU → A100

In [ ]:
import torch

print("="*60)
print("🔧 GPU CHECK")
print("="*60)

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    print(f"\n✅ GPU Available: {gpu_name}")
    print(f"   Memory: {gpu_mem:.1f} GB")
    
    if 'A100' in gpu_name:
        print("\n🚀 A100 detected - OPTIMAL!")
    elif 'T4' in gpu_name:
        print("\n⚠️ T4 detected - Reduce batch size to 8-16")
else:
    print("\n❌ NO GPU! Go to: Runtime → Change runtime type → GPU")

print("\n" + "="*60)

## Step 3: Discover Your Data Folders

Let's find where your GEE exports went on Drive!

In [ ]:
import os
from pathlib import Path

# Base Drive path
DRIVE_ROOT = "/content/drive/MyDrive"

print("📂 Searching for California Fire Model folders...\n")

# Find all folders containing 'California_Fire_Model'
cali_folders = []
for item in os.listdir(DRIVE_ROOT):
    if 'California_Fire_Model' in item or 'california' in item.lower():
        full_path = os.path.join(DRIVE_ROOT, item)
        if os.path.isdir(full_path):
            cali_folders.append(item)
            
            # Check for tif files
            tif_count = len(list(Path(full_path).rglob("*.tif")))
            print(f"   📁 {item}: {tif_count} .tif files")

print(f"\n📊 Found {len(cali_folders)} folders")

## Step 4: Configure Your Folder Paths

**⚠️ EDIT THIS CELL based on what you see above!**

In [ ]:
# =========================================================
# 🔧 CONFIGURATION
# =========================================================

DRIVE_ROOT = "/content/drive/MyDrive"

# Fire names to look for in folder names
FIRE_FOLDERS = [
    "dixie",
    "caldor", 
    "creek",
    "camp",
    "mendocino",
    "thomas",
    "kincade",
    "woolsey",
]

# Name of the code folder (has .py files)
CODE_FOLDER_NAME = "California_Fire_Model"

print("✅ Configuration set!")

In [ ]:
# Auto-detect folders based on naming pattern
print("🔍 Auto-detecting data folders...\n")

fire_data_folders = []
healthy_folders = []
code_folder = None

for item in os.listdir(DRIVE_ROOT):
    full_path = os.path.join(DRIVE_ROOT, item)
    if not os.path.isdir(full_path):
        continue
    
    item_lower = item.lower()
    
    # Check if it ends with "healthy" or contains " healthy"
    # This catches "California_Fire_Model healthy" pattern
    if item_lower.endswith('healthy') or ' healthy' in item_lower:
        healthy_folders.append(full_path)
        print(f"   🌲 Healthy: {item}")
        continue
    
    # Check if it's a fire data folder (contains fire name)
    is_fire_folder = False
    for fire in FIRE_FOLDERS:
        if fire.lower() in item_lower:
            # Make sure it's not the base code folder
            if item != CODE_FOLDER_NAME:
                fire_data_folders.append(full_path)
                print(f"   🔥 Fire: {item}")
                is_fire_folder = True
            break
    
    if is_fire_folder:
        continue
    
    # Check if it's the code folder (exact match, has .py files)
    if item == CODE_FOLDER_NAME:
        try:
            has_code = any(f.endswith('.py') for f in os.listdir(full_path) if os.path.isfile(os.path.join(full_path, f)))
            if has_code:
                code_folder = full_path
                print(f"   📁 Code: {item}")
        except:
            pass

print(f"\n📊 Found:")
print(f"   Fire folders: {len(fire_data_folders)}")
print(f"   Healthy folders: {len(healthy_folders)}")
print(f"   Code folder: {'✅' if code_folder else '❌'}")

## Step 5: Copy Data to Local SSD

**This makes training 10-50x faster!**

In [ ]:
import shutil
import time

# Local paths (fast SSD)
LOCAL_DATA = "/content/local_data"
LOCAL_FIRES = f"{LOCAL_DATA}/fires"
LOCAL_HEALTHY = f"{LOCAL_DATA}/healthy"
LOCAL_CODE = "/content/California-Fire-Model"

# Create directories
os.makedirs(LOCAL_FIRES, exist_ok=True)
os.makedirs(LOCAL_HEALTHY, exist_ok=True)

print("="*60)
print("📦 COPYING DATA TO LOCAL SSD")
print("="*60)

total_copied = 0
start_time = time.time()

# Copy fire data
print("\n🔥 Copying fire data...")
for src_folder in fire_data_folders:
    folder_name = os.path.basename(src_folder)
    
    # Extract fire name from folder
    fire_name = None
    for fire in FIRE_FOLDERS:
        if fire.lower() in folder_name.lower():
            fire_name = fire
            break
    
    if fire_name:
        dest_folder = f"{LOCAL_FIRES}/{fire_name}"
    else:
        dest_folder = f"{LOCAL_FIRES}/{folder_name}"
    
    os.makedirs(dest_folder, exist_ok=True)
    
    # Copy all tif files
    tif_files = list(Path(src_folder).rglob("*.tif"))
    for tif_file in tif_files:
        dest_file = os.path.join(dest_folder, tif_file.name)
        if not os.path.exists(dest_file):
            shutil.copy2(tif_file, dest_file)
        total_copied += 1
    
    print(f"   ✅ {folder_name}: {len(tif_files)} files → {fire_name or folder_name}/")

# Copy healthy data
print("\n🌲 Copying healthy data...")
for src_folder in healthy_folders:
    folder_name = os.path.basename(src_folder)
    
    # Copy all tif files to healthy folder
    tif_files = list(Path(src_folder).rglob("*.tif"))
    for tif_file in tif_files:
        dest_file = os.path.join(LOCAL_HEALTHY, tif_file.name)
        if not os.path.exists(dest_file):
            shutil.copy2(tif_file, dest_file)
        total_copied += 1
    
    print(f"   ✅ {folder_name}: {len(tif_files)} files")

# Copy code
print("\n📁 Copying code...")
if code_folder and os.path.exists(code_folder):
    if os.path.exists(LOCAL_CODE):
        shutil.rmtree(LOCAL_CODE)
    shutil.copytree(code_folder, LOCAL_CODE)
    print(f"   ✅ Code copied to {LOCAL_CODE}")
else:
    print(f"   ⚠️ Code folder not found. Check CODE_FOLDER_NAME variable.")

elapsed = time.time() - start_time
print(f"\n✅ Copied {total_copied} files in {elapsed:.1f} seconds")
print("="*60)

In [ ]:
# Verify the local data structure
print("📂 Local Data Structure:\n")

# Check fires
print(f"🔥 {LOCAL_FIRES}/")
if os.path.exists(LOCAL_FIRES):
    for fire_dir in sorted(os.listdir(LOCAL_FIRES)):
        fire_path = os.path.join(LOCAL_FIRES, fire_dir)
        if os.path.isdir(fire_path):
            count = len([f for f in os.listdir(fire_path) if f.endswith('.tif')])
            print(f"   └── {fire_dir}/: {count} tiles")

# Check healthy
print(f"\n🌲 {LOCAL_HEALTHY}/")
if os.path.exists(LOCAL_HEALTHY):
    count = len([f for f in os.listdir(LOCAL_HEALTHY) if f.endswith('.tif')])
    print(f"   └── {count} tiles")

# Check code
print(f"\n📁 {LOCAL_CODE}/")
if os.path.exists(LOCAL_CODE):
    py_files = [f for f in os.listdir(LOCAL_CODE) if f.endswith('.py')]
    print(f"   └── {len(py_files)} Python files")
else:
    print("   └── ❌ Not found!")

## Step 6: Install Dependencies

In [ ]:
!pip install -q rasterio albumentations tqdm

print("✅ Dependencies installed!")

## Step 7: Final Verification

In [ ]:
import sys
sys.path.insert(0, LOCAL_CODE)

print("="*60)
print("✅ FINAL VERIFICATION")
print("="*60)

checks = []

# GPU
gpu_ok = torch.cuda.is_available()
gpu_name = torch.cuda.get_device_name(0) if gpu_ok else "None"
checks.append(('GPU', gpu_ok, gpu_name))

# Fire data
fire_count = len(list(Path(LOCAL_FIRES).rglob("*.tif"))) if os.path.exists(LOCAL_FIRES) else 0
checks.append(('Fire data', fire_count > 50, f"{fire_count} tiles"))

# Healthy data  
healthy_count = len(list(Path(LOCAL_HEALTHY).rglob("*.tif"))) if os.path.exists(LOCAL_HEALTHY) else 0
checks.append(('Healthy data', healthy_count > 0, f"{healthy_count} tiles"))

# Code
code_ok = os.path.exists(f"{LOCAL_CODE}/config.py")
checks.append(('Code files', code_ok, LOCAL_CODE if code_ok else "Not found"))

# Imports
try:
    from config import TRAINING_CONFIG
    import_ok = True
except Exception as e:
    import_ok = False
checks.append(('Config imports', import_ok, "OK" if import_ok else str(e)[:30]))

print("\n   Check                 Status   Details")
print("   " + "-"*55)
for name, ok, details in checks:
    status = "✅" if ok else "❌"
    print(f"   {name:<20} {status}       {details}")

all_ok = all(ok for _, ok, _ in checks)
print("\n" + "="*60)

if all_ok:
    print("🚀 ALL CHECKS PASSED! Ready to train!")
    print("\n📋 Next: Open 02_colab_train_a100.ipynb")
    print("\n📂 Important paths:")
    print(f"   Data: {LOCAL_DATA}")
    print(f"   Code: {LOCAL_CODE}")
else:
    print("⚠️ Some checks failed. Fix the issues above.")

print("="*60)

---

## ✅ Setup Complete!

**If all checks passed:**
1. Open `02_colab_train_a100.ipynb`
2. Start training!

**Local paths:**
- Fires: `/content/local_data/fires/`
- Healthy: `/content/local_data/healthy/`
- Code: `/content/California-Fire-Model/`

**Note:** If Colab disconnects, re-run this setup notebook.